In [1]:
!pip3 install pyspark

  Using cached https://files.pythonhosted.org/packages/9e/b6/6a4fb90cd235dc8e265a6a2067f2a2c99f0d91787f06aca4bcf7c23f3f80/py4j-0.10.9-py2.py3-none-any.whl


In [61]:
from pyspark.sql import SparkSession
import os

# set environment variable PYSPARK_SUBMIT_ARGS
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/phani/Desktop/elasticsearch-hadoop-7.9.2/dist/elasticsearch-hadoop-7.9.2.jar pyspark-shell'

spark = SparkSession \
    .builder \
    .getOrCreate()
sc = spark.sparkContext
df = spark.read.csv("employee.csv", header=True)
print(df.show())

+------+-----------+----------+--------------+---------+------+--------------------+----------------+------------------+--------------------+-------------+-------------+-----------+--------------+---------------+------------------+---------------+---------------+----------------+---------------------+-----------+--------------+--------------+---------+----------------------+------+-----------+-----------+------------+-------------+--------------------+-------------+-----+-----+---------+-----------+---------------+
|Emp ID|Name Prefix|First Name|Middle Initial|Last Name|Gender|              E Mail|   Father's Name|     Mother's Name|Mother's Maiden Name|Date of Birth|Time of Birth|Age in Yrs.|Weight in Kgs.|Date of Joining|Quarter of Joining|Half of Joining|Year of Joining|Month of Joining|Month Name of Joining|Short Month|Day of Joining|DOW of Joining|Short DOW|Age in Company (Years)|Salary|Last % Hike|        SSN|  Phone No. |   Place Name|              County|         City|State|  Z

In [ ]:
df.select('Date of Joining')

In [64]:
df = df.fillna(0)
display(df)

DataFrame[Emp ID: string, Name Prefix: string, First Name: string, Middle Initial: string, Last Name: string, Gender: string, E Mail: string, Father's Name: string, Mother's Name: string, Mother's Maiden Name: string, Date of Birth: string, Time of Birth: string, Age in Yrs.: string, Weight in Kgs.: string, Date of Joining: string, Quarter of Joining: string, Half of Joining: string, Year of Joining: string, Month of Joining: string, Month Name of Joining: string, Short Month: string, Day of Joining: string, DOW of Joining: string, Short DOW: string, Age in Company (Years): string, Salary: string, Last % Hike: string, SSN: string, Phone No. : string, Place Name: string, County: string, City: string, State: string, Zip: string, Region: string, User Name: string, Password: string]

In [65]:
df.dtypes

[('Emp ID', 'string'),
 ('Name Prefix', 'string'),
 ('First Name', 'string'),
 ('Middle Initial', 'string'),
 ('Last Name', 'string'),
 ('Gender', 'string'),
 ('E Mail', 'string'),
 ("Father's Name", 'string'),
 ("Mother's Name", 'string'),
 ("Mother's Maiden Name", 'string'),
 ('Date of Birth', 'string'),
 ('Time of Birth', 'string'),
 ('Age in Yrs.', 'string'),
 ('Weight in Kgs.', 'string'),
 ('Date of Joining', 'string'),
 ('Quarter of Joining', 'string'),
 ('Half of Joining', 'string'),
 ('Year of Joining', 'string'),
 ('Month of Joining', 'string'),
 ('Month Name of Joining', 'string'),
 ('Short Month', 'string'),
 ('Day of Joining', 'string'),
 ('DOW of Joining', 'string'),
 ('Short DOW', 'string'),
 ('Age in Company (Years)', 'string'),
 ('Salary', 'string'),
 ('Last % Hike', 'string'),
 ('SSN', 'string'),
 ('Phone No. ', 'string'),
 ('Place Name', 'string'),
 ('County', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Zip', 'string'),
 ('Region', 'string'),
 ('User Name

In [66]:
df1 =  df.select('Emp ID', 'First Name', 'Middle Initial', 'Gender', 'E Mail', 'Salary', 'County', 'City', 'State', 'Region', regexp_replace(col("Last % Hike"), "%", "").alias("Hike"))
df1.show()

+------+----------+--------------+------+--------------------+------+--------------------+-------------+-----+---------+----+
|Emp ID|First Name|Middle Initial|Gender|              E Mail|Salary|              County|         City|State|   Region|Hike|
+------+----------+--------------+------+--------------------+------+--------------------+-------------+-----+---------+----+
|742048|    Lizeth|             P|     F|lizeth.mccoll@ibm...|147446|               Stark|     Alliance|   OH|  Midwest|  14|
|671135| Argentina|             O|     F|argentina.hern@nt...|129174|District of Columbia|   Washington|   DC|    South|   8|
|965851|    Damian|             N|     M|damian.patillo@ou...|158746|              Fresno|       Burrel|   CA|     West|   8|
|224660|   Imogene|             P|     F|imogene.hagopian@...| 55761|              Brazos|        Bryan|   TX|    South|  14|
|852694|    Walker|             E|     M|walker.wallach@ao...|197519|              Vernon|     Newllano|   LA|    Sout

In [67]:
#df1.select("Last % Hike", regexp_replace(col("Last % Hike"), "%", "").alias("Hike")).show()

In [68]:
df1.printSchema()

root
 |-- Emp ID: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Hike: string (nullable = true)



In [71]:
df1 = df1.withColumn("Salary",df1["Salary"].cast('int'))
df1 = df1.withColumn("Hike",df1["Hike"].cast('int'))

In [72]:
df1.printSchema()

root
 |-- Emp ID: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Hike: integer (nullable = true)



#  Count number of employees in each Region, country, city

In [73]:
df1.select("Region").distinct().show()

+---------+
|   Region|
+---------+
|  Midwest|
|    South|
|     West|
|Northeast|
+---------+



### number of employees based on Region 

In [74]:
df1.groupBy('Region').count().show()

+---------+-------+
|   Region|  count|
+---------+-------+
|  Midwest|1372439|
|    South|1855656|
|     West| 884921|
|Northeast| 886984|
+---------+-------+



### Number of employees based on Country

In [75]:
df1.groupBy('County').count().show()

+--------------------+-----+
|              County|count|
+--------------------+-----+
|               Bucks| 7697|
|            Thurston| 2875|
|             Palermo|  155|
|           Worcester|13022|
|              Grimes|  872|
|               Tyler| 1740|
|             Hanover| 1347|
|                Utah| 2777|
|      East Feliciana|  688|
|              Aitkin|  918|
|        Saint George|  109|
|Skagway-Hoonah-An...|  995|
|            Bluffton|  113|
|           Petroleum|  258|
|          Prattville|  118|
|          Charleston| 4334|
|               Izard| 1707|
|              Hawaii| 3775|
|               Pasco| 3564|
|            Montcalm| 2052|
+--------------------+-----+
only showing top 20 rows



### Number of employees based on city

In [76]:
df1.groupBy('City').count().show()

+-----------------+-----+
|             City|count|
+-----------------+-----+
|          Hanover| 2314|
|          Palermo|  466|
|        Fairbanks| 1296|
|        Worcester| 2181|
|Kingsford Heights|  131|
|     Plaucheville|  157|
|         Bluffton|  820|
|           Grimes|  254|
|    Harrison City|  122|
|      Piney Creek|  110|
|   Camp H M Smith|  131|
|       Prattville|  487|
|      Santa Paula|  220|
|            Kiana|  112|
|         Cheriton|  138|
|         Fredonia|  920|
|            Tyler| 1999|
|         Thurston|  352|
|          Jemison|  104|
|       Sugar City|  228|
+-----------------+-----+
only showing top 20 rows



# orderby gender and salary

In [77]:
gender = df1.orderBy(df1.Gender)
gender.show()

+------+----------+--------------+------+--------------------+------+------------+------------------+-----+---------+----+
|Emp ID|First Name|Middle Initial|Gender|              E Mail|Salary|      County|              City|State|   Region|Hike|
+------+----------+--------------+------+--------------------+------+------------+------------------+-----+---------+----+
|418550|  Vivienne|             Z|     F|vivienne.adair@sb...|198240|   Jefferson|        Louisville|   KY|    South|  17|
|663429|    Tambra|             S|     F|tambra.knutson@mi...|145873|    Mackinac|   Mackinac Island|   MI|  Midwest|   1|
|675785|    Nguyet|             H|     F|nguyet.mraz@yahoo...| 47744| Mecklenburg|         Charlotte|   NC|    South|  10|
|281267|     Treva|             L|     F|treva.hitz@hotmai...| 85286|        Erie|           Buffalo|   NY|Northeast|   7|
|300443|     Kathi|             V|     F|  kathi.teal@aol.com|155262|   St. Clair|              Yale|   MI|  Midwest|  26|
|158748| Steffan

In [78]:
print("Total no of male employees")
gender.filter(gender.Gender == 'M').count()

Total no of male employees


2498868

In [79]:
print("Total no of Female employees")
gender.filter(gender.Gender == 'F').count()

Total no of Female employees


2501132

## Order by salary and gender

In [80]:
df1.orderBy(df1.Gender, df1.Salary).show()

+------+----------+--------------+------+--------------------+------+----------+-------------+-----+---------+----+
|Emp ID|First Name|Middle Initial|Gender|              E Mail|Salary|    County|         City|State|   Region|Hike|
+------+----------+--------------+------+--------------------+------+----------+-------------+-----+---------+----+
|616703|      Amee|             J|     F|amee.gagliano@yah...| 40000|    Dallas|       Dallas|   TX|    South|  14|
|813535|  Lorraine|             Z|     F|lorraine.latorre@...| 40000|  Converse|     Glenrock|   WY|     West|  21|
|741049|      Kari|             C|     F|kari.osman@yahoo.com| 40000|     Mingo|      Justice|   WV|    South|  29|
|676813|    Nannie|             Q|     F|nannie.sansom@gma...| 40000|    Bergen|       Mahwah|   NJ|Northeast|  12|
|828071| Madelaine|             U|     F|madelaine.barrett...| 40000|     Aiken|        Aiken|   SC|    South|  26|
|479980|     Chong|             A|     F|chong.manthey@yah...| 40000|   

##  Salary wise sort employee data


In [81]:
df1.sort("Salary").select('Emp ID', 'Salary', 'County').show()

+------+------+-----------+
|Emp ID|Salary|County     |
+------+------+-----------+
|479980|40000 |Jackson    |
|525554|40000 |Rock       |
|272737|40000 |Johnson    |
|502767|40000 |St. Joseph |
|227163|40000 |Windsor    |
|832197|40000 |McHenry    |
|200251|40000 |Winnebago  |
|374296|40000 |Shawano    |
|828071|40000 |Aiken      |
|631239|40000 |Graham     |
|314186|40000 |Lyon       |
|336973|40000 |Ouachita   |
|117353|40000 |Potter     |
|140140|40000 |Routt      |
|571129|40000 |Larimer    |
|680938|40000 |Somerset   |
|631239|40000 |Hoke       |
|511018|40000 |Franklin   |
|964794|40000 |Westchester|
|506893|40000 |Evangeline |
+------+------+-----------+
only showing top 20 rows



In [82]:
print("Average salary of a employee is ")
df1.agg({'Salary': 'avg'}).show()

Average salary of a employee is 
+--------------+
|   avg(Salary)|
+--------------+
|119992.4947336|
+--------------+



In [83]:
print("Maximum salary of a employee is ")
df1.agg({'Salary': 'max'}).show()

Maximum salary of a employee is 
+-----------+
|max(Salary)|
+-----------+
|     200000|
+-----------+



In [84]:
print("Minimum salary of a employee is ")
df1.agg({'Salary': 'min'}).show()

Minimum salary of a employee is 
+-----------+
|min(Salary)|
+-----------+
|      40000|
+-----------+



## Employee Summary table 

In [87]:
summary = df1.select('Emp ID', 'Salary', 'Hike', 'Region')
summary.show()

+------+------+----+---------+
|Emp ID|Salary|Hike|   Region|
+------+------+----+---------+
|742048|147446|  14|  Midwest|
|671135|129174|   8|    South|
|965851|158746|   8|     West|
|224660| 55761|  14|    South|
|852694|197519|  24|    South|
|144102|103839|   6|     West|
|687793| 60101|   5|  Midwest|
|636308|115639|  26|  Midwest|
|218660|126048|  19|Northeast|
|465691|193757|  28|Northeast|
|242339| 43826|  14|  Midwest|
|636949| 84036|   2|  Midwest|
|890565|182148|  30|     West|
|206270|181592|  26|Northeast|
|418390|131922|  23|  Midwest|
|359884|184946|   1|  Midwest|
|301840|103833|  27|     West|
|232212|103292|  10|     West|
|450156| 66545|   7|    South|
|158031| 41892|   1|  Midwest|
+------+------+----+---------+
only showing top 20 rows



In [88]:
summary.describe().show()

+-------+------------------+-----------------+-----------------+-------+
|summary|            Emp ID|           Salary|             Hike| Region|
+-------+------------------+-----------------+-----------------+-------+
|  count|           5000000|          5000000|          5000000|5000000|
|   mean|    555602.1214034|   119992.4947336|       14.9987616|   null|
| stddev|256579.70180242613|46176.82746950901|8.943850740305255|   null|
|    min|            111111|            40000|                0|Midwest|
|    max|            999999|           200000|               30|   West|
+-------+------------------+-----------------+-----------------+-------+



In [94]:
print("Total employees whose salary is greater than average salary")
summary.filter(summary['Salary']>119992).count()


Total employees whose salary is greater than average salary


2500407

In [95]:
print("Total employees whose got hike above average value")
summary.filter(summary['Hike']>14.9987616).count()

Total employees whose got hike above average value


2580138

In [96]:
print("Total employees whose salary is below average salary")
summary.filter(summary['Salary']<119992).count()

Total employees whose salary is below average salary


2499556

In [97]:
print("Total employees whose got hike below average value")
summary.filter(summary['Hike']<14.9987616).count()

Total employees whose got hike below average value


2419862

In [100]:
#Employess from different regions 
Midwest = summary.filter(summary['Region'] == 'Midwest').count()
print("Total Employees from Midwest: ", Midwest)

South = summary.filter(summary['Region'] == 'South').count()
print("Total Employees from South: ", South)

West = summary.filter(summary['Region'] == 'West').count()
print("Total Employees from West: ", West)

Northeast = summary.filter(summary['Region'] == 'Northeast').count()
print("Total Employees from Northeast: ", Northeast)

Total Employees from Midwest:  1372439
Total Employees from South:  1855656
Total Employees from West:  884921
Total Employees from Northeast:  886984


In [17]:
# es_write_conf = {
#         "es.nodes" : "localhost",
#         "es.port" : "9200",
#         "es.resource" : 'employee/emp',
#         "es.input.json": "yes",
#         "es.mapping.id": "doc_id"
#     }
       
# rdd.saveAsNewAPIHadoopFile(
#         path='-',
#      outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat", keyClass="org.apache.hadoop.io.NullWritable",
#         valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
#         conf=es_write_conf)




# # df1.write.format("org.elasticsearch.spark.sql")
# .option("es.resource", "employee/emp")
# .option("es-port", "9200")
# .option("es.nodes" : "localhost").save()